In [ ]:
# Colab Cell 1: Install Unsloth & QLoRA Dependencies (quiet mode)

# 1. Upgrade pip quietly
!pip install -q --upgrade pip

# 2. Install Unsloth with Colab extras quietly
!pip install -q --upgrade unsloth

# 3. Install core libraries without re-pulling extras, in quiet mode
!pip install -q --no-deps \
    peft>=0.7.0 \
    trl>=0.4.0 \
    accelerate>=0.25.0 \
    datasets>=2.15.0 \
    scipy>=1.11.0 \
    huggingface_hub>=0.19.0


In [ ]:
# Colab Cell 2: Hugging Face Login

from huggingface_hub import login
login()  # follow the popup to enter your token securely


In [ ]:
# Colab Cell 3: Load,ize, Inject LoRA & Train with Unsloth on T4 GPU

import torch
import json
import os
import gc
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
import psutil

# ── Configuration ───────────────────────────────────────────────────────────
MODEL_NAME   = "Qwen/Qwen2.5-3B-Instruct"
OUTPUT_DIR   = "qwen-algospeak-unsloth"
DATASET_FILE = "training_dataset_colab.json"
MAX_SEQ_LEN  = 512

# ── 1. Load & Inspect Dataset ───────────────────────────────────────────────
def load_large_dataset(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dataset not found: {path}")
    with open(path, "r") as f:
        data = json.load(f)
    print(f"✅ Loaded {len(data):,} samples")
    algospeak = sum(1 for x in data if x.get("is_algospeak", False))
    print(f"📊 Algospeak samples: {algospeak:,} ({algospeak/len(data)*100:.1f}%)")
    return data

training_data = load_large_dataset(DATASET_FILE)

# ── 2. 4-bit Model Load via Unsloth ──────────────────────────────────────────
print("🤖 Loading & quantizing model in 4-bit mode…")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    load_in_4bit=True,
    max_seq_length=MAX_SEQ_LEN,
    device_map="auto",
    trust_remote_code=True,
)

# ── 3. Inject LoRA Adapters ─────────────────────────────────────────────────
print("🔧 Applying LoRA adapters…")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)
model.print_trainable_parameters()

# ── 4. Prepare & Tokenize Dataset ──────────────────────────────────────────
def format_prompt(sample):
    return f"""### Instruction:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}"""

texts = [format_prompt(s) for s in training_data]
dataset = Dataset.from_dict({"text": texts}).train_test_split(test_size=0.1, seed=42)

# Cleanup large lists
del training_data, texts
gc.collect()
torch.cuda.empty_cache()

def tokenize_fn(examples):
    out = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=MAX_SEQ_LEN,
    )
    out["labels"] = out["input_ids"].copy()
    return out

print("🔤 Tokenizing dataset…")
tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"],
    batch_size=1000,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

# ── 5. TrainingArguments & Trainer Setup ────────────────────────────────────
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    # report_to="none",           # UNCOMMENT this line to disable WandB tracking
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    args=training_args,
    data_collator=data_collator,
)

# ── 6. Start Training ───────────────────────────────────────────────────────
print("🚀 Starting QLoRA training on massive Algospeak dataset…")
trainer.train()


In [ ]:
# Colab Cell 6: Post-Training Model Export & 4-bit GGUF Conversion
print("💾 Creating 4-bit GGUF model...")

# ── Configuration ───────────────────────────────────────────────────────────
ADAPTER_DIR = "qwen-algospeak-unsloth"
GGUF_DIR = f"{ADAPTER_DIR}_gguf_4bit"

# ── 1. Save LoRA Adapters (Small ~100MB) ─────────────────────────────────────
print("📁 Saving LoRA adapters...")
model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print(f"✅ LoRA adapters saved to {ADAPTER_DIR}")

# ── 2. Convert to 4-bit GGUF Only ────────────────────────────────────────────
print("⚙️ Converting to 4-bit GGUF...")
os.makedirs(GGUF_DIR, exist_ok=True)

# ONLY 4-bit GGUF (dynamic quantization)
model.save_pretrained_gguf(
    f"{GGUF_DIR}/q4_k_m", 
    tokenizer, 
    quantization_method="q4_k_m"
)
print("✅ 4-bit GGUF created")

# ── 3. Memory Cleanup ──────────────────────────────────────────────────────────
print("🧹 Cleaning up memory...")
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("✅ 4-bit GGUF model ready for download!")


In [ ]:
# Colab Cell 7: Download LoRA Adapters & 4-bit GGUF
from google.colab import files
import subprocess
import os

# ── Configuration ───────────────────────────────────────────────────────────
ADAPTER_DIR = "qwen-algospeak-unsloth"
GGUF_DIR = f"{ADAPTER_DIR}_gguf_4bit"

# Zip file names
ADAPTER_ZIP = "lora_adapters.zip"           # ~100MB
GGUF_4BIT_ZIP = "gguf_4bit_model.zip"       # ~2GB

# ── 1. Zip LoRA Adapters ─────────────────────────────────────────────────────
if os.path.isdir(ADAPTER_DIR):
    print(f"📦 Zipping LoRA adapters: {ADAPTER_DIR} → {ADAPTER_ZIP}")
    subprocess.run(["zip", "-r", ADAPTER_ZIP, ADAPTER_DIR], check=True)
    size_mb = os.path.getsize(ADAPTER_ZIP) / (1024*1024)
    print(f"📊 Adapter size: {size_mb:.1f} MB")
else:
    raise FileNotFoundError(f"❌ Adapter folder not found: {ADAPTER_DIR}")

# ── 2. Zip 4-bit GGUF Model ──────────────────────────────────────────────────
if os.path.isdir(GGUF_DIR):
    print(f"📦 Zipping 4-bit GGUF: {GGUF_DIR} → {GGUF_4BIT_ZIP}")
    subprocess.run(["zip", "-r", GGUF_4BIT_ZIP, GGUF_DIR], check=True)
    size_gb = os.path.getsize(GGUF_4BIT_ZIP) / (1024*1024*1024)
    print(f"📊 4-bit GGUF size: {size_gb:.1f} GB")
else:
    print(f"⚠️ No 4-bit GGUF found at {GGUF_DIR}")

# ── 3. Download Both Files ───────────────────────────────────────────────────
print("\n🎯 Downloading:")
print("1. 📱 LoRA Adapters (~100MB)")
print("2. 🚀 4-bit GGUF Model (~2GB) - Ready for Ollama/LM Studio")

print("\n⬇️ Starting downloads...")
files.download(ADAPTER_ZIP)
if os.path.exists(GGUF_4BIT_ZIP):
    files.download(GGUF_4BIT_ZIP)

print("✅ Downloads complete! 🎉")
